<font size = "6"> Fordham Sports Analytics Society Big Data Bowl 2023 - Data Preparation </font>

<font size = "4"> Prepare data sets provided in the case for our exploratory work and model building. </font>

- Authors:  Peter Majors, Chris Orlando, Jack Townsend, and Etienne Busnel
- Kaggle:  https://www.kaggle.com/competitions/nfl-big-data-bowl-2023/overview (Resources)
- Our Github:  https://github.com/peterlmajors/FSAS_BigDataBowl_2023 (Up-To-Date Code)

<font size="5"> Importing And Merging Original Data</font>

In [162]:
#Import Required Packages
import pandas as pd
import numpy as np
import math

#Notebook Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [163]:
#Importing Kaggle Data

#Games - Basic Information On All Games
games = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/games.csv")

#pffScout - PFF Judgements For Each Player On Each Play
pffScout = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/pffScoutingData.csv")

#Players - Basics On Players
players = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/players.csv")

#Plays - Everthing About Specific Plays
plays = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/plays.csv")

#Week - Frame-By-Frame Player Tracking
week1 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week1.csv")
week2 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week2.csv")
week3 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week3.csv")
week4 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week4.csv")
week5 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week5.csv")
week6 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week6.csv")
week7 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week7.csv")
week8 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/case_data/week8.csv")

#Import PFF Data On QB Pressures
pff_qb_pressure = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/pff_qb_pressure.csv")

In [164]:
#Merging All Data Together (Not Needed If Importing Merged DataFrames In Next Cell)

#Merge Game Info And Play Info, Each Play Now Has More Context
pbp = pd.merge(games, plays, how = "inner", on = "gameId")

# #Append All Week Dataframes #Prefer For Weeks To Stay As Separate
week = pd.DataFrame()
weeks = [week1, week2, week3, week4, week5, week6, week7, week8]
for i in range(0,len(weeks)):
    week = week.append(weeks[i])

# #Merge Player Tracking, PFF Grading, and Player History Data
ptrack = pd.merge(pffScout, week, how = "inner", on = ["gameId", "playId", "nflId"])
ptrack = pd.merge(ptrack, players, how = "inner", on = ["nflId"])

#Export pbp Data Frame For Safe Keeping
pbp.to_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/pbp.csv")

C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\1762755123.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week = week.append(weeks[i])
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\1762755123.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week = week.append(weeks[i])
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\1762755123.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week = week.append(weeks[i])
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\1762755123.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  week = week.append(weeks[i])
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\1762755123.py:10: FutureWar

<font size="5"> Find Distance of Each Player From The Quarterback </font>

In [165]:
#Find QB Distance Tracking and Merge Onto ptrack DataFrame (Distance As Yards)
ptrack_qb = ptrack[ptrack.pff_positionLinedUp == "QB"]
ptrack_qb = ptrack_qb[['gameId', 'playId', 'frameId', 'x','y', 's', 'a', 'dis','o','dir']]
ptrack = pd.merge(ptrack, ptrack_qb, how = 'inner', on = ['gameId', 'playId', 'frameId'])
ptrack = ptrack.rename(columns = {"x_x":"x", "y_x": "y", "s_x": "s", "a_x": "a", "dis_x": "dis", "o_x": "o", "dir_x": "dir",
                       "x_y":"x_qb", "y_y": "y_qb", "s_y": "s_qb", "a_y": "a_qb", "dis_y": "dis_qb", "o_y": "o_qb", "dir_y": "dir_qb"})
ptrack['dist_from_qb'] = np.hypot((ptrack.x - ptrack.x_qb), (ptrack.y - ptrack.y_qb))

#Remove NaN Values From ptrack
ptrack = ptrack.fillna(0)

<font size="5"> Find Angle of Each Player From The Quarterback and Orientation Deviation From That Angle</font>

In [166]:
#Calculate Angles Between Players And QB (Angles Work Same As Orientation and Direction Metrics From Kaggle)
ptrack['angle_to_qb'] = round((90 - np.degrees(np.arctan2(((ptrack.y_qb - ptrack.y)**2), (ptrack.x_qb - ptrack.x)**2))),2)

#Adjust For Quadrants Relative To QB Position On The Field
ptrack.loc[(ptrack.y < ptrack.y_qb) & (ptrack.x > ptrack.x_qb), 'angle_to_qb'] = ptrack.angle_to_qb #Bottom Right
ptrack.loc[(ptrack.y < ptrack.y_qb) & (ptrack.x < ptrack.x_qb), 'angle_to_qb'] = ptrack.angle_to_qb #Bottom Left
ptrack.loc[(ptrack.y > ptrack.y_qb) & (ptrack.x < ptrack.x_qb), 'angle_to_qb'] = ptrack.angle_to_qb #Top Left

#Deviation Between Orientation of Player And Their Angle To The Quarterback
#Evaluating Offensive Linemen, This Means We Want To Measure If Their Backs Are To The QB They're Protecting
#Positive = Player's Left Shoulder Turned Towards QB So Many Degrees // Negative = Player's Right Shoudler Turned Towards QB So Many Degrees
ptrack['angle_to_qb_diff_o'] = ptrack.o - ptrack.angle_to_qb

#Change QB Angles To Themselves To Null
ptrack.loc[ptrack['pff_positionLinedUp'] == 'QB', 'angle_to_qb'] = np.nan
ptrack.loc[ptrack['pff_positionLinedUp'] == 'QB', 'angle_to_qb_diff_o'] = np.nan


<font size = '5'> Determine When Pass Rushers Enter Immediate Zone On Known Blocking Plays (First Block In Play) </font>

<b> Immediate Zone Depth </b> - 1.5 Yard

<b> Immediate Zone Width </b> - 2 Yards

In [167]:
#Find All Tracking Data For All Frames Where Blocks Occur Between Rushers and Passers

#Create Data Frame With Only Pass Blocking Plays
ptrack_block = ptrack.loc[ptrack['pff_blockType'] != 0]

#Find Rows With Players Who Were Blocked Against (And Who Have The Role of Pass Rusher)
ptrack_block_rushers = ptrack.loc[(ptrack.nflId.isin(ptrack_block.pff_nflIdBlockedPlayer)) & (ptrack.pff_role == "Pass Rush")]
ptrack_block_rushers = ptrack_block_rushers[['gameId', 'playId', 'nflId', 'frameId', 'pff_role', 'pff_positionLinedUp', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'displayName']]

#Merge Pass Blocking Plays Data With Pass Rusher Data
ptrack_imm_box = ptrack_block.merge(ptrack_block_rushers, left_on = ['gameId', 'playId', 'pff_nflIdBlockedPlayer', 'frameId'], right_on = ['gameId', 'playId', 'nflId', 'frameId'], how = 'inner')

#Reduce To Columns Of Interest
ptrack_imm_box = ptrack_imm_box[['gameId', 'playId', 'nflId_x', 'frameId', 'pff_role_x', 'pff_positionLinedUp_x', 'pff_blockType','x_x', 'y_x', 's_x', 'a_x', 'dis_x', 'o_x', 'dir_x', 'displayName_x','nflId_y', 'pff_role_y', 'pff_positionLinedUp_y', 'x_y', 'y_y', 's_y', 'a_y', 'dis_y', 'o_y', 'dir_y', 'displayName_y']]

#Rename Columns
ptrack_imm_box = ptrack_imm_box.rename(columns = {"nflId_x":"nflId_blocker", "displayName_x": "displayName_blocker", "pff_role_x": "pff_role_blocker", "pff_positionLinedUp_x": "pff_positionLinedUp_blocker", "x_x": "x_blocker", "y_x": "y_blocker", "s_x": "s_blocker", "a_x": "a_blocker", "dis_x": "dis_blocker", "o_x": "o_blocker", "dir_x": "dir_blocker", "nflId_y":"nflId_rusher", "displayName_y": "displayName_rusher", "pff_role_y": "pff_role_rusher", "pff_positionLinedUp_y": "pff_positionLinedUp_rusher", "x_y": "x_rusher", "y_y": "y_rusher", "s_y": "s_rusher", "a_y": "a_rusher", "dis_y": "dis_rusher", "o_y": "o_rusher", "dir_y": "dir_rusher"})

#Calculate Distance Between Pass Blocker And Pass Rusher at Each Frame
ptrack_imm_box['blocker_rusher_distance'] = round(np.hypot((ptrack_imm_box.y_blocker - ptrack_imm_box.y_rusher), (ptrack_imm_box.x_blocker - ptrack_imm_box.x_rusher)),2)

#Calculate Difference Between Rusher Direction and Blocker Orientation
ptrack_imm_box['diff_btw_rusher_dir_blocker_o'] = abs((ptrack_imm_box.dir_rusher - 180) - ptrack_imm_box.o_blocker)


In [168]:
#Define Function To Determine If The Pass Rusher Is In The Zone On Each Play

def rusher_in_imm_zone(o_line,x_line,y_line,x_rush,y_rush): #o_line Refers To Orientation Of Pass Blocker

    length = 1.5 #Depth of The Immediate Zone
    width = 2 #Width Of The Immediate Zone

    o_line_rad = math.radians(o_line)
    in_zone = 0

    resx_1 = x_line - .5 * width * math.cos(o_line_rad)
    resx_2 = resx_1 + length * math.sin(o_line_rad)
    resx_3 = x_line + .5 * width * math.cos(o_line_rad)
    resx_4 = x_line + .5 * width * math.cos(o_line_rad) + length * math.sin(o_line_rad)

    resy_3 = y_line - .5 * width * math.sin(o_line_rad)
    resy_4 = resy_3 + length * math.cos(o_line_rad)
    resy_1 = y_line + .5 * width * math.sin(o_line_rad)
    resy_2 = y_line + .5 * width * math.sin(o_line_rad) + length * math.cos(o_line_rad)
    
    x_min = min(resx_1,resx_2,resx_3,resx_4)
    x_max = max(resx_1,resx_2,resx_3,resx_4)

    if ((x_min <= x_rush) & (x_max >= x_rush)):
        y_to_x_dict = {resy_1:resx_1, resy_2:resx_2, resy_3:resx_3, resy_4:resx_4}
        x_to_y_dict = {resx_1:resy_1, resx_2:resy_2, resx_3:resy_3, resx_4:resy_4}
        y_max = max(resy_1,resy_2,resy_3,resy_4)
        y_min = min(resy_1,resy_2,resy_3,resy_4)

        x_top = y_to_x_dict[y_max]
        x_bottom = y_to_x_dict[y_min]
        y_dec, y_max_at_x_rush, y_min_at_x_rush = 0, 0, 0

        if (o_line%90 == 0): #Avoid Undefined Slope Errors, Dictionary With Multiple Keys
            y_max_at_x_rush = y_max
            y_min_at_x_rush = y_min

        else:
            #Find Max Value Rusher's y Coordinate Can Be In Zone
            if x_top >= x_rush:
                slope = (y_max - x_to_y_dict[x_min]) / (x_top - x_min) 
                y_int_top = y_max - slope*x_top
                y_max_at_x_rush = slope*x_rush + y_int_top

            else:
                slope = (x_to_y_dict[x_max] - y_max) / (x_max - x_top)
                y_int_top = y_max - slope*x_top
                y_int_bottom = y_min - slope*x_bottom
                y_max_at_x_rush = slope*x_rush + y_int_top

            #Find Min Value Rusher's y Coordinate Can Be In Zone
            if x_bottom >= x_rush:
                slope2 = (y_min - x_to_y_dict[x_min]) / (x_bottom - x_min)
                y_int_bottom = y_min - slope2*x_bottom
                y_min_at_x_rush = slope2*x_rush + y_int_bottom

            else:
                slope2 = (x_to_y_dict[x_max] - y_min) / (x_max - x_bottom)
                y_int_bottom = y_min - slope2*x_bottom
                y_min_at_x_rush = slope2*x_rush + y_int_bottom

        #If Rusher y In Between Above Two Values, They Are In The Immediate Zone
        if ((y_rush >= y_min_at_x_rush) & (y_rush <= y_max_at_x_rush)):
            in_zone = 1

    return in_zone 

In [169]:
#Binary Column Created Determining If A Rusher Is In A Pass Blocker's Immediate Zone
for i in range(len(ptrack_imm_box)):
    ptrack_imm_box.at[i, "rusher_in_imm_box"] = rusher_in_imm_zone(ptrack_imm_box.at[i, "o_blocker"], ptrack_imm_box.at[i, "x_blocker"], ptrack_imm_box.at[i, "y_blocker"], ptrack_imm_box.at[i, "x_rusher"], ptrack_imm_box.at[i, "y_rusher"])

In [170]:
#Merge The ptrack_imm_box Data Frame Onto ptrack
ptrack_imm_box = ptrack_imm_box[['gameId','playId','nflId_blocker','frameId', 'nflId_rusher', 'displayName_rusher', 'pff_positionLinedUp_rusher', 'x_rusher',	'y_rusher', 's_rusher', 'a_rusher', 'dis_rusher', 'o_rusher', 'dir_rusher', 'rusher_in_imm_box', 'blocker_rusher_distance', 'diff_btw_rusher_dir_blocker_o']]
ptrack = ptrack.merge(ptrack_imm_box, left_on = ['gameId','playId','nflId','frameId'], right_on = ['gameId','playId','nflId_blocker','frameId'], how = 'left')

<font size = '5'> Filter ptrack To Frames Only Occuring During QB Possesion Of The Football, Creating ptrack_qb_poss </font>

<b> Starting At</b> - ball_snap     

<b> Ending At</b> - qb_sack, qb_strip_sack, run, pass_forward (In That Order)

In [171]:
#Establish The Frame In Each Play At Which The Starting/Ending Of QB Possession Occurs With New Columns (Long Run-Time)

#Temporary Data Frame
ptrack_qb_poss = ptrack

#Arrays of Starting and Ending Values
poss_events = ['ball_snap', 'autoevent_ballsnap','pass_forward', 'autoevent_passforward', 'run', 'qb_sack', 'qb_strip_sack']

#Add Frames Of Events To Each Frame of Each Plays
for i in range(len(poss_events)):
    temp = ptrack.loc[ptrack.event == poss_events[i]][['gameId', 'playId', 'frameId']].drop_duplicates()
    ptrack_qb_poss = ptrack_qb_poss.merge(temp, on = ['gameId', 'playId'], how = "left")

#Rename Columns
indices_to_be_changed = [58, 59, 60, 61, 62, 63, 64, 15]
names_to_insert = ["ball_snap_event_frame", "autoevent_ballsnap_event_frame", "pass_forward_event_frame", "autoevent_passforward_event_frame", 
                   "run_event_frame", "qb_sack_event_frame", "qb_strip_sack_event_frame", "frameId"]
for i in range(len(indices_to_be_changed)):
    ptrack_qb_poss.columns.values[indices_to_be_changed[i]] = names_to_insert[i]


C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\3798676709.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'frameId_x'} in the result is deprecated and will raise a MergeError in a future version.
  ptrack_qb_poss = ptrack_qb_poss.merge(temp, on = ['gameId', 'playId'], how = "left")
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\3798676709.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'frameId_x'} in the result is deprecated and will raise a MergeError in a future version.
  ptrack_qb_poss = ptrack_qb_poss.merge(temp, on = ['gameId', 'playId'], how = "left")
C:\Users\Peter\AppData\Local\Temp\ipykernel_3636\3798676709.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'frameId_x'} in the result is deprecated and will raise a MergeError in a future version.
  ptrack_qb_poss = ptrack_qb_poss.merge(temp, on = ['gameId', 'playId'], how = "left")


In [172]:
#Reduce 'ball_snap', 'autoevent_ballsnap','pass_forward', and 'autoevent_passforward' To Just Two Columns

#If Any Column of The Four of Interest Is Null, Fill Them So They Can Be Compared
poss_events_to_be_condensed = ['ball_snap_event_frame', "autoevent_ballsnap_event_frame", "pass_forward_event_frame", "autoevent_passforward_event_frame"]
for i in range(len(poss_events_to_be_condensed)):
    if i <= 1: #For Ball Snap, Fill With A Frame Greater Than The Max In Any Play (Since We Want The Lesser)
        ptrack_qb_poss[poss_events_to_be_condensed[i]] = ptrack_qb_poss[poss_events_to_be_condensed[i]].fillna(250)
    else: #For Pass Forward, Fill With A Frame Lesser Than The Min In Any Play (Since We Want Greater Than)
        ptrack_qb_poss[poss_events_to_be_condensed[i]] = ptrack_qb_poss[poss_events_to_be_condensed[i]].fillna(-1)
    
#Determine Earlier of "ball_snap_event_frame" and "autoevent_ballsnap_event_frame", Smaller Of The Two Into "ball_snap_event_frame_" (Doesn't Affect Calculations Much & More Time The Better)
ptrack_qb_poss["ball_snap_event_frame_"] = np.where(ptrack_qb_poss["ball_snap_event_frame"] <= ptrack_qb_poss["autoevent_ballsnap_event_frame"], ptrack_qb_poss["ball_snap_event_frame"], ptrack_qb_poss["autoevent_ballsnap_event_frame"])

#Determine Earlier of "pass_forward_event_frame" and "autoevent_passforward_event_frame", Larger Of The Two Into "pass_forward_event_frame_" (Expect Rushers To Not Give Up Until Ball Fully Out of Pocket)
ptrack_qb_poss["pass_forward_event_frame_"] = np.where(ptrack_qb_poss["pass_forward_event_frame"] >= ptrack_qb_poss["autoevent_passforward_event_frame"], ptrack_qb_poss["pass_forward_event_frame"], ptrack_qb_poss["autoevent_passforward_event_frame"])

#When Both Auto and Non-Auto Columns Are Null, Set Their Values Appropriately
ptrack_qb_poss.loc[ptrack_qb_poss["ball_snap_event_frame_"] == 250, "ball_snap_event_frame_"] = 1 #If No Ball Snap Is Recorded, Set QB Possesion Starting at Frame 1
ptrack_qb_poss.loc[ptrack_qb_poss["pass_forward_event_frame_"] == -1, "pass_forward_event_frame_"] = np.nan #If No Pass Forward Is Recorded, Don't Make It End Of Possession

#Remove The Four Uneccessary Columns and Rename The Two Now In Their Place
ptrack_qb_poss = ptrack_qb_poss.drop(['ball_snap_event_frame', 'autoevent_ballsnap_event_frame', 'pass_forward_event_frame','autoevent_passforward_event_frame'], axis = 1)
ptrack_qb_poss = ptrack_qb_poss.rename(columns = {"ball_snap_event_frame_": "ball_snap_event_frame", "pass_forward_event_frame_": "pass_forward_event_frame"})

In [173]:
#Filter Out Rows That Don't Satisfy QB Possesion Conditions

#Drop All That Come BEFORE Ball Snap
ptrack_qb_poss = ptrack_qb_poss.drop(ptrack_qb_poss[ptrack_qb_poss.frameId < ptrack_qb_poss.ball_snap_event_frame].index) 

#Drop All That Come AFTER QB Sack, QB Strip Sack, Run, Pass Forward In That Order (If The Event Isn't In The Play Nothing Will Be Dropped)

#If A QB Is Sacked, There's Nothing More A Line Can Do (Although Sometimes The QB Can Get The Pass Off). That's Why Strip & Normal Sacks Are The First Two.
#If A QB Runs And Then Throws, We Say That He's Too Far Away For Linemen For It To Be Fair. If Pass Then Run, Count The Pass Frame First.

ending_event_frames = ['qb_sack_event_frame', 'qb_strip_sack_event_frame', 'run_event_frame', 'pass_forward_event_frame']
for i in range(len(ending_event_frames)):
    ptrack_qb_poss = ptrack_qb_poss.drop(ptrack_qb_poss[ptrack_qb_poss['frameId'] > ptrack_qb_poss[ending_event_frames[i]]].index) 

In [174]:
#Add In A Beginning Frame and Ending Frame Column For Each Play, In Addition To The Event Frames (Since Plays Can Have Multiple "Possession Ending" Events)
frame_min = ptrack_qb_poss.groupby(['gameId', 'playId'])['frameId'].min().reset_index()
frame_max = ptrack_qb_poss.groupby(['gameId', 'playId'])['frameId'].max().reset_index()

#Rename The Frames 
frame_min = frame_min.rename(columns = {'frameId': 'frame_first'})
frame_max = frame_max.rename(columns = {'frameId': 'frame_last'})

#Merge The Frames Onto ptrack_qb_poss
ptrack_qb_poss = ptrack_qb_poss.merge(frame_min, on = ['gameId', 'playId'], how = "inner")
ptrack_qb_poss = ptrack_qb_poss.merge(frame_max, on = ['gameId', 'playId'], how = "inner")

<font size = '5'> Filter Data To Exclude Block Types, Positions, and Columns That Clutter The Data </font>

In [175]:
#Remove Columns: run_event_frame, qb_sack_event_frame, qb_strip_sack_event_frame, pass_forward_event_frame
ptrack_qb_poss = ptrack_qb_poss.pff_blockType.drop('BH', 'CH', 'NB', 'SR')

ptrack_qb_poss = ptrack_qb_poss.drop(columns = ['run_event_frame','qb_sack_event_frame', 'qb_strip_sack_event_frame', 'pass_forward_event_frame', 'ball_snap_event_frame'])

<font size="5"> Export Player Tracking Data Frame With New Fields </font>

In [176]:
#Export Distance Data Frame To .csv (Long Run Time)
# ptrack.to_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/ptrack.csv")
ptrack_qb_poss.to_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/merged_data/ptrack_qb_poss.csv")